<a href="https://colab.research.google.com/github/Zelechos/Generador-Css/blob/main/model/GeneradorCSS-GPT-2XL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto Generador de CSS [v0.2.0]**

---



# Clonamos nuestro repositorio en donde se encuentra nuestro dataset con este comando

In [2]:
!git clone https://github.com/Zelechos/Generador-Css.git

Cloning into 'Generador-Css'...
remote: Enumerating objects: 700, done.
remote: Counting objects: 100% (299/299), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 700 (delta 172), reused 276 (delta 151), pack-reused 401
Receiving objects: 100% (700/700), 52.77 MiB | 35.93 MiB/s, done.
Resolving deltas: 100% (358/358), done.


### en este caso no es necesario ya que tenemos el dataset en github por ende tenemos que trabajar con el dataset que esta en formato json

### Accedemos a los datos del dataset TEST.json


In [3]:
import json

with open('/content/Generador-Css/dataset/TEST.json') as file:
    data = json.load(file)

#accedemos a la los datos
data_styles = data["styles"]
print(data_styles[0])


#accediendo a los estilos
styles = data_styles[0]["style"]
print("\nstyles access => ",styles)


#accediendo a los selectores
selector = data_styles[0]["selector"]
print("\nselector access => ",selector)


{'style': 'body { overflow:hidden ; background:rgb(25,35,125) } ', 'selector': 'body'}

styles access =>  body { overflow:hidden ; background:rgb(25,35,125) } 

selector access =>  body


### Implementamos nuestro modelo Pre-entrenado GPT-2 XL

In [4]:
from transformers import GPT2Tokenizer, TFGPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
model = TFGPT2Model.from_pretrained('gpt2-xl')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [5]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2-xl')
set_seed(42)
generator("The man worked as a", max_length=10, num_return_sequences=5)

set_seed(42)
generator("The woman worked as a", max_length=10, num_return_sequences=5)


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The woman worked as a supervisor for the company.'},
 {'generated_text': "The woman worked as a volunteer at the Children's"},
 {'generated_text': 'The woman worked as a waitress at a New Jersey'},
 {'generated_text': 'The woman worked as a lab technician but quit to'},
 {'generated_text': 'The woman worked as a prostitute, according to reports'}]

Analisis del Modelo GTP-2 XL

**Experimento 001**
- Cambiamos el parametro max_length a 50
que dadas las observaciones define la longitud de palabras maximas que puede tener la sentencia a generar
- El parametro num_return_sequences = 1

In [6]:
set_seed(42)
generator("Alan Turing was a", max_length=50, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Alan Turing was a brilliant mathematician, but you can\'t beat a human being in a chess match with any of the chess engines out there. So for computer chess, he said, "You are only ever going to beat a human being in a game'}]

**Experimento 002**
- Cambiamos el parametro max_length a 2
que dadas las observaciones define la longitud de palabras maximas que puede tener la sentencia a generar
- Tambien Cambiamos el parametro num_return_sequences a 3
que dadas las observaciones es la cantidad de sentencias que genera el modelo considerando la longitud de palabras antes definida

*Se observo que al poner el parametro de max_length = 2 presenta un error por la longitud del input*

In [7]:
generator("a Transformer in AI is", max_length=2, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ValueError: Input length of input_ids is 2, but `max_length` is set to 2. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

**Experimento 003**
- Cambiamos el parametro max_length a 7
que dadas las observaciones define la longitud de palabras maximas que puede tener la sentencia a generar incluyendo el input
- Tambien Cambiamos el parametro num_return_sequences a 3
que dadas las observaciones es la cantidad de sentencias que genera el modelo considerando la longitud de palabras antes definida

Se observo que con el parametro max_length = 2 hay una error dado que el parametro max_length debe superar en longitud de input_ids es N entonces el parametro max_length siempre tiene que ser almenos N + 1 o mayor que.

In [11]:
generator("a Transformer in AI is", max_length=10, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'a Transformer in AI is the same as that'},
 {'generated_text': 'a Transformer in AI is a powerful engine ('},
 {'generated_text': 'a Transformer in AI is more than worth it'}]

**Experimento 004**
- Definimos el parametro de la semilla en 30 para observar que sucede


In [12]:
set_seed(30)
generator("a Artificial Neural Network is", max_length=15, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'a Artificial Neural Network is a machine learning algorithm that combines a number of techniques'},
 {'generated_text': 'a Artificial Neural Network is a program capable of learning and recognizing symbols and patterns'},
 {'generated_text': 'a Artificial Neural Network is trained on a large number of digitized music examples'}]